# Algorithm Comparison: Thistlethwaite vs Kociemba vs Korf

**Author:** Alex Toska  
**Affiliation:** University of Patras  
**Phase:** 9 (Demos & UI Visualization)  

---

## Overview

This notebook provides a comprehensive comparison of the three implemented solving algorithms using the Phase 8 evaluation framework.

We'll compare:
- **Solution quality** (number of moves)
- **Speed** (computation time)
- **Memory usage**
- **Success rates**

## Setup

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.cube.rubik_cube import RubikCube
from src.evaluation.algorithm_comparison import AlgorithmComparison

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful!")

## Part 1: Single Scramble Comparison

Let's compare all three algorithms on a single scramble.

In [ ]:
# Create scramble
cube = RubikCube()
cube.scramble(moves=10, seed=42)
scramble = getattr(cube, '_scramble_moves', [])

print(f"Scramble: {' '.join(scramble)}")
print(f"Depth: {len(scramble)} moves\n")

# Initialize comparison framework
comparison = AlgorithmComparison(
    thistlethwaite_timeout=30.0,
    kociemba_timeout=60.0,
    korf_timeout=120.0,
    korf_max_depth=20
)

# Run comparison
result = comparison.compare_on_scramble(cube, scramble_id=0)

In [ ]:
# Display results in a table
data = []
for name, algo_result in [
    ("Thistlethwaite", result.thistlethwaite),
    ("Kociemba", result.kociemba),
    ("Korf IDA*", result.korf)
]:
    if algo_result.solved:
        data.append({
            "Algorithm": name,
            "Solved": "✓",
            "Moves": algo_result.solution_length,
            "Time (s)": f"{algo_result.time_seconds:.3f}",
            "Memory (MB)": f"{algo_result.memory_mb:.2f}"
        })
    else:
        data.append({
            "Algorithm": name,
            "Solved": "✗",
            "Moves": "-",
            "Time (s)": f"{algo_result.time_seconds:.3f}",
            "Memory (MB)": f"{algo_result.memory_mb:.2f}"
        })

df = pd.DataFrame(data)
print("\n=== Comparison Results ===")
print(df.to_string(index=False))

## Part 2: Batch Comparison

Test all algorithms on multiple scrambles for statistical analysis.

In [ ]:
# Run batch test
print("Running batch test on 10 scrambles...\n")

results = comparison.run_batch_test(
    n_scrambles=10,
    scramble_depth=10,
    seed=42
)

print("\n✓ Batch test complete!")

In [ ]:
# Generate summary statistics
summaries = comparison.generate_summary()

# Display summary
summary_data = []
for name in ['Thistlethwaite', 'Kociemba', 'Korf_IDA*']:
    s = summaries[name]
    summary_data.append({
        "Algorithm": name.replace('_', ' '),
        "Success Rate": f"{s.success_rate * 100:.1f}%",
        "Avg Moves": f"{s.avg_solution_length:.1f}",
        "Avg Time (s)": f"{s.avg_time_seconds:.3f}",
        "Avg Memory (MB)": f"{s.avg_memory_mb:.2f}"
    })

summary_df = pd.DataFrame(summary_data)
print("\n=== Summary Statistics ===")
print(summary_df.to_string(index=False))

## Part 3: Visualizations

In [ ]:
# Plot solution length comparison
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

algorithms = ['Thistlethwaite', 'Kociemba', 'Korf_IDA*']
labels = ['Thistlethwaite', 'Kociemba', 'Korf IDA*']
colors = ['#3498db', '#2ecc71', '#9b59b6']

# Solution length
moves = [summaries[alg].avg_solution_length for alg in algorithms]
ax1.bar(labels, moves, color=colors, alpha=0.7)
ax1.set_ylabel('Moves')
ax1.set_title('Average Solution Length')
ax1.grid(axis='y', alpha=0.3)

# Time
times = [summaries[alg].avg_time_seconds for alg in algorithms]
ax2.bar(labels, times, color=colors, alpha=0.7)
ax2.set_ylabel('Seconds')
ax2.set_title('Average Solving Time')
ax2.grid(axis='y', alpha=0.3)

# Memory
memory = [summaries[alg].avg_memory_mb for alg in algorithms]
ax3.bar(labels, memory, color=colors, alpha=0.7)
ax3.set_ylabel('MB')
ax3.set_title('Average Memory Usage')
ax3.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Winner Analysis

In [ ]:
print("🏆 Winner Analysis\n")

# Fewest moves
moves_data = [(name.replace('_', ' '), summaries[name].avg_solution_length) 
              for name in algorithms]
winner = min(moves_data, key=lambda x: x[1])
print(f"Fewest Moves: {winner[0]} ({winner[1]:.1f} avg)")

# Fastest
time_data = [(name.replace('_', ' '), summaries[name].avg_time_seconds) 
             for name in algorithms]
winner = min(time_data, key=lambda x: x[1])
print(f"Fastest: {winner[0]} ({winner[1]:.3f}s avg)")

# Least memory
mem_data = [(name.replace('_', ' '), summaries[name].avg_memory_mb) 
            for name in algorithms]
winner = min(mem_data, key=lambda x: x[1])
print(f"Least Memory: {winner[0]} ({winner[1]:.2f} MB avg)")

# Best success rate
success_data = [(name.replace('_', ' '), summaries[name].success_rate) 
                for name in algorithms]
winner = max(success_data, key=lambda x: x[1])
print(f"Best Success Rate: {winner[0]} ({winner[1] * 100:.1f}%)")

## Conclusions

### Key Findings

1. **Thistlethwaite** is the fastest but produces sub-optimal solutions
2. **Kociemba** offers the best balance of speed and solution quality
3. **Korf IDA*** finds optimal solutions but can be slower

### Trade-offs

| Algorithm | Best For |
|-----------|----------|
| Thistlethwaite | Quick demos, educational purposes |
| Kociemba | Practical solving, competitions |
| Korf IDA* | Research, optimal solutions |

### Next Steps

- Try different scramble depths (5, 10, 15, 20)
- Increase sample size for better statistics
- Compare with Phase 8 comprehensive results
- Experiment with different timeout settings

## Export Results

Uncomment and run to export results.

In [ ]:
# Export to JSON
# comparison.export_results('../results/notebook_comparison.json')

# Export summary to Markdown
# comparison.export_summary_table('../results/notebook_summary.md', format='markdown')

print("Export complete (if uncommented)!")